In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../i4c/")

import disturbances
from data.buildings.test.sfh_84_94_soc import sfh_84_94_soc
from data.buildings.test.sfh_84_94_ad_ref import sfh_84_94_ad_ref

# Data Engineering

This Jupyter notebook was used to merge the the price and weather data. Unfortunetely, the price data is licences and therefore not public. The weather data was obtained from https://ec.europa.eu/jrc/en/PVGIS/tools/tmy

# Merge Price Files

In [ ]:
all_eval_prices  = pd.DataFrame()

In [ ]:
eval_prices = pd.read_csv('', header=None)
eval_prices['timestamp'] = pd.date_range("", periods=8760, freq="H")

In [ ]:
all_eval_prices = all_eval_prices.append(eval_prices)

In [ ]:
all_eval_prices.to_csv('')

# Preprocess data from

https://ec.europa.eu/jrc/en/PVGIS/tools/tmy

In [ ]:
YEARS = [2010, 2011, 2012, 2013, 2014, 2015, 2016]
TEST_YEAR = [2016]
EVAL_YEAR = [2010]
TRAIN_YEAR = [2010, 2011, 2012, 2013, 2014, 2015]
FILE_WITH_GAINS = ''

FILE_WITHOUT_GAINS = ''

In [ ]:
weather_data = pd.read_csv(FILE_WITH_GAINS, skiprows=8)
weather_data = weather_data.dropna()
weather_data['Time'] = pd.to_datetime(weather_data['time'], format='%Y%m%d:%H%M')
weather_data = weather_data.rename(columns={"T2m": "T_amb"})
weather_data['Time'] = weather_data['Time'] - datetime.timedelta(minutes=10)
weather_data = weather_data[weather_data['Time'].dt.year.isin(YEARS)]
weather_data = weather_data.reset_index()

weather_data_g = pd.read_csv(FILE_WITHOUT_GAINS, skiprows=8)
weather_data_g = weather_data_g.dropna()
weather_data_g['Time'] = pd.to_datetime(weather_data_g['time'], format='%Y%m%d:%H%M')
weather_data_g = weather_data_g.rename(columns={"T2m": "T_amb"})
weather_data_g['Time'] = weather_data_g['Time'] - datetime.timedelta(minutes=10)
weather_data_g = weather_data_g[weather_data_g['Time'].dt.year.isin(YEARS)]
weather_data_g = weather_data_g.reset_index()

weather_data['G(i)'] = weather_data_g['G(i)']

### Merge with electricity stock price data

In [ ]:
e_prices = pd.read_csv('', parse_dates=['timestamp'])
e_prices = e_prices[e_prices['timestamp'].dt.year.isin(YEARS)]
weather_data['EEX'] = e_prices['0']

weather_data.to_csv('')

### Drop Test Data

In [ ]:
weather_data[weather_data['Time'].dt.year.isin(TEST_YEAR)].to_csv('')

### Drop Eval Data


In [ ]:
eval_data = weather_data[weather_data['Time'].dt.year.isin([2013])]
eval_data = eval_data[eval_data['Time'].dt.month.isin([10,11,12])]
eval_data.to_csv('')

### Drop Train Data

In [ ]:
weather_data[weather_data['Time'].dt.year.isin(TRAIN_YEAR)].to_csv('')

# Add gains profile for robustness testing

Caution, this was not included in the master thesis but was already provided for future works.

In [ ]:
COLS_TO_USE = ['Time', 'EEX', 'T_amb', 'Gb(i)', 'Gd(i)', 'Gr(i)', 'G(i)']
test_data = pd.read_csv('', parse_dates=['Time'])
test_data = test_data[COLS_TO_USE]

solar_df = pd.DataFrame({'dhi': test_data['Gd(i)'], 
                         'ghi': test_data['G(i)'], 
                         'dni': test_data['Gb(i)'], 
                         'time': test_data['Time']
                        }).set_index('time')

test_data['gains'] = disturbances.get_solar_gains(solar_df, sfh_84_94_soc).values
test_data['blow_air'] = np.random.choice(a=[False, True], size=(len(test_data)), p=[0.97, 0.03])

In [ ]:
test_data.set_index('Time')[0:100]['gains'].plot()

In [ ]:
test_data.to_csv('')